# [10] Data Setup for Keras GPU
This file functions similarly to "[0] Data Setup" but reworked to create images to run into the Keras Model in "[11] Keras Testing Safe vs Malignant". The idea was that if I wanted different resolutions of the concat_imgs array I could create them here but it ended up being best for GPU VRAM to just use the lower res (162\*2, 135) concat images that we've already been using (plus this makes this model more comparable to the others). This means that nothing from this notebook is actually used in the final keras model notebook.

In [1]:
## Import libraries

%matplotlib inline
 
import matplotlib.pyplot as plt
import numpy as np
import os
import pprint
pp = pprint.PrettyPrinter(indent=4)
import seaborn as sns

import pickle
import random
import math as m
import time
import pandas as pd
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
              tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        
from skimage.transform import resize
import mnist
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [2]:
## loading in important variables

# Keys
with open('OFFICIAL_keys.pkl', 'rb') as f:
    keys = pickle.load(f)
train_set = keys[:int(0.8*len(keys))]
test_set = keys[int(0.8*len(keys)):int(0.9*len(keys))+1]
validation_set = keys[int(0.9*len(keys))+1:]

# Classifications
with open('classifications.pkl', 'rb') as f:
    classifications = pickle.load(f)
classes = []
for i in range(2575):
    if classifications[2*i] != classifications[2*i+1]:
        print("Classification not equal")
    classes.append(classifications[2*i])
classes = np.array(classes)

# Abnormalities
with open('abnormalities.pkl', 'rb') as f:
    abnormalities = pickle.load(f)
abnorms = []
for i in range(2575):
    if abnormalities[2*i] != abnormalities[2*i+1]:
        print("abnormalities not equal")
    abnorms.append(abnormalities[2*i])
abnorms = np.array(abnorms)

# Orientations
with open('orientations.pkl', 'rb') as f:
    orientations = pickle.load(f)
orientations = np.array(orientations)    
scale = (725, 605)

In [3]:
## Functions for rescaling images
def rescale_images(input_images, scale):
    new_list = []
    for image in input_images:
        new_list.append(resize(image, scale))
    return new_list
def np_rescale_images(input_images, scale):
    new_array = np.empty((input_images.shape[0], scale[0], scale[1]))
    for i, image in enumerate(input_images):
        new_array[i] = resize(image, scale)
    return new_array
def scale_resolution(input_resolution, scale):
    yi = input_resolution[0]
    xi = input_resolution[1]
    pixels = xi*yi
    ar = yi/xi
    scale_pixels = pixels*scale
    xo = (m.sqrt(scale_pixels/ar))
    yo = (xo*ar)
    return [int(round(yo, 0)), int(round(xo, 0))]

In [6]:
## This function will splits the images into three groups side view, bottom view, and a combined concatenated view
## Takes (index, row, col) returns three np arrays in with images in flattened form -
## with shape (index, image)
def create_side_bottom_concat(np_imgs, to_print=False, orients=orientations):
    side=[]
    below=[]
    flattened_imgs = np_imgs.reshape(np_imgs.shape[0], np_imgs.shape[1]*np_imgs.shape[2])
    if to_print:
        print(flattened_imgs.shape)
    for i in range(0, flattened_imgs.shape[0]):
        if len(orients[i]) == 2:
            side.append(flattened_imgs[i])
        else:
            below.append(flattened_imgs[i])   
    side = np.array(side)
    below = np.array(below)
    concat = np.concatenate((side, below), 1)
    if to_print:
        print('\nconcatenated images')
        print(f'concat.shape={concat.shape}')
        print('\nside images')
        print(f'side.shape={side.shape}')
        print('\nbelow images')
        print(f'below.shape={below.shape}')
    return side, below, concat

In [7]:
## This function will flip the right side images to the left for better data workability
## Takes (index, row, col) and returns (index, row, col)
def flip_list(np_imgs, to_print=False):
    flip_list=[]
    for q in range(0,len(np_imgs)):
        if orientations[q]=='L' or orientations[q]=='FL': #flipping the right images
            flip_list.append(np.flip(np_imgs[q],1)) #the integer there is the axis being flipped
        else:
            flip_list.append(np_imgs[q])
    np_flip_list = np.array(flip_list)
    
    if to_print:
        print(f'len(np_imgs)={len(np_imgs)}')
        print(f'len(orientations)={len(orientations)}')
        print('list of flipped images')
        print(f'len(flip_list)={len(flip_list)}')
        print(f'flip_list[0].shape={flip_list[0].shape}')
        print(f"np_flip_list.shape={np_flip_list.shape}")
    return np_flip_list

In [8]:
## Set New Scale
original_scale = [2294, 1914]
new_scale = scale_resolution(original_scale, 0.01)
print(new_scale)

[229, 191]


In [9]:
## Set Raw Images
with open('images.pkl', 'rb') as f:
    raw_images = pickle.load(f)

In [10]:
## Scale Raw Images Down and Convert to Numpy Array
scaled_images = np.array(rescale_images(raw_images, new_scale))

In [11]:
## Save finished input set
with open(f'scaled_images_{new_scale[0]}_{new_scale[1]}.pkl', 'wb') as f:
    pickle.dump(scaled_images, f) 

In [12]:
del raw_images

In [19]:
## Reload Scaled Images
with open(f'scaled_images_{new_scale[0]}_{new_scale[1]}.pkl', 'rb') as f:
    scaled_images = pickle.load(f)

In [13]:
## Flip images to all be on the same side
flipped_scaled_images = flip_list(scaled_images)

In [14]:
scaled_side, scaled_below, scaled_concat = create_side_bottom_concat(flipped_scaled_images)

In [15]:
del scaled_images, flipped_scaled_images, scaled_side, scaled_below

In [16]:
## Format Images for Keras
scaled_concat = np.reshape(scaled_concat, (scaled_concat.shape[0], new_scale[0]*2, new_scale[1]))
scaled_concat = (scaled_concat / 255) - 0.5
scaled_concat = np.expand_dims(scaled_concat, axis=3)

In [17]:
## Save finished input set
with open(f'finished_concat_images_{new_scale[0]}_{new_scale[1]}.pkl', 'wb') as f:
    pickle.dump(scaled_concat, f) 